# Doppler imaging

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import starry
import numpy as np

## Star with a grey spot

Let's instantiate a ``DopplerMap``. In addition to the spherical harmonic degree, we need to provide a bit more information when instantiating a Doppler map. This includes the number of epochs we're modeling (``nt``), the number of spectral components (``nc``), the number of wavelength bins in the model (``nw``), and the endpoints of the wavelength grid in nanometers (``wav1`` and ``wav2``).

In [ ]:
map = starry.DopplerMap(15, nt=20, nc=1, nw=199, wav1=642.75, wav2=643.25)

Specifically, we're modeling a degree `15` spherical harmonic map with `20` epochs (i.e., the number of spectra we've observed and would like to model), one spectral component (we'll talk about this below), and `199` wavelength bins between `642.75` and `643.25` nanometers (spanning the wavelength of the classical Fe I line used in Doppler imaging).

Let's specify two more properties relevant to Doppler imaging: the stellar inclination and the equatorial velocity.

In [ ]:
map.inc = 60
map.veq = 30000

By default, these are in degrees and meters per second, respectively.

The final thing we must do is specify what the surface of the star looks like, both spatially *and* spectrally. The spatial map can be specified by setting spherical harmonic coefficients or by directly loading a latitude-longitude map:

In [ ]:
map.load("spot", force_psd=True)

where ``force_psd`` ensures the spherical harmonic expansion is positive-semidefinite (i.e., non-negative everywhere). The spectrum can be specified directly as an array evaluated on the wavelength grid ``map.wavs``. Let's add a single narrow Gaussian absorption line at the central wavelength:

In [ ]:
map.spectrum = 1.0 - 0.75 * np.exp(-0.5 * (map.wavs - 643.0) ** 2 / 0.025 ** 2)

Our model is now fully specified. Before we compute the model, it's convenient to visualize what our star looks like in practice. This is done via the ``map.show()`` method:

In [ ]:
map.show()

The map at the top left shows the surface of the star at a specific wavelength, indicated by the vertical orange line in the plot below it. You can change the wavelength by scrolling over the map. As you move in and out of the absorption line, notice how the entire star gets dimmer and brighter, respectively. You can also move the mouse around the map to visualize the local spectrum at different regions of the surface. That's what the plot below is showing: the spectrum emerging from a *specific* point on the surface.

The map at the top right shows the surface of the star as seen on the sky (not at any specific wavelength $-$ just the normalized average of all the spectral components). That map is there so you can rotate it (by scrolling) and see how the *observed spectrum* changes over time. That's the black curve in the plot below it. For reference, the orange curve shows the spectrum one would observe in the absence of any Doppler shifts $-$ i.e., in the limit that $v\sin i = 0$.

In addition to the Doppler broadening due to the fast rotation of the star, there are wiggles in the spectral line caused by the dark spots coming in and out of view. This is the classical Doppler imaging signal: these wiggles contain lots of information about the full spectro-spatial map of the star.

The ``show`` method above implicitly computed the Doppler imaging model for us, but in most cases we would like to access it directly. As with regular ``starry`` maps, this is done by calling ``flux()``:

In [ ]:
theta = np.linspace(0, 360, 20)
flux = map.flux(theta)

This method takes one positional argument: the angular phase of the star, ``theta``. Note that ``theta`` must be an array of length equal to ``map.nt``, the number of epochs we told ``starry`` about earlier. The method returns a two-dimensional array of fluxes at each wavelength (or, alternatively, spectra at each point in time):

In [ ]:
flux.shape

In this case, that's ``20`` spectra, one at each phase ``theta``, each containing ``199`` wavelength bins. The wavelength grid for the flux is given by ``map.wavf``. Let's visualize our model:

In [ ]:
plt.plot(map.wavf, flux.T, color="C0", lw=1, alpha=0.5)
plt.xlabel("wavelength (nm)")
plt.ylabel("flux");

The baseline for each epoch is slightly different, because the total flux from the star changes as spots come in and out of view. Spectroscopic measurements aren't usually sensitive to this $-$ they're usually measurements *relative* to an unknown baseline. We can normalize by the baseline at each epoch to look at these relative changes:

In [ ]:
plt.plot(map.wavf, (flux / flux[:, 0].reshape(-1, 1)).T, color="C0", lw=1, alpha=0.5)
plt.xlabel("wavelength (nm)")
plt.ylabel("normalized flux");

## Star with a spectrally variable spot

The previous example showed a star with a single spectral component (``nc = 1``). This means there is only a single spectrum: it's the same everywhere on the surface, up to an overall scaling (given by the intensity of the spatial map at every point). The ``SPOT`` feature was therefore "grey": it simply dimmed the outgoing flux equally at all wavelengths.

In reality, spots have different spectra than the rest of the stellar surface. They're at a different temperature, so their blackbody function is different, but they could also contain different species with distinct absorption lines. To model this with ``starry``, we can create a Doppler map with multiple spectral components. The best way to understand this is by example, so let's create a map with ``4`` distinct components:

In [ ]:
map = starry.DopplerMap(15, nt=20, nc=4)
map.inc = 60
map.veq = 30000

Each component has its own associated spatial map. Let's load four different maps, one for each component. To make things fun, we'll make each component one of the letters in the word ``SPOT``. This time the ``SPOT`` will be bright (just to mix things up), but otherwise it's similar to the one in the previous example.

In [ ]:
map.load(["s", "p", "o", "t"], force_psd=True)

Each component also has its own associated spectrum. Let's give each one a spectrum with a single absorption line, each at a different wavelength:

In [ ]:
mu = np.array([642.69, 642.90, 643.10, 643.31])
sig = 0.025
dw = map.wavs.reshape(-1, 1) - mu.reshape(1, -1)
map.spectrum = 1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2)

As before, let's visualize the map:

In [ ]:
map.show()

Things got a lot more complicated! Spend some time interacting with the figure to get a handle on what's going on.